## Restaurant Practice 2 ##

Dataset: restaurant_transactions.xlsx

In [48]:
## Step 1: Import Dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [49]:
## Step 2: Load Data

df = pd.read_excel("restaurant_transactions.xlsx")

print(f"Shape of data: {df.shape}")
print(df.head())

Shape of data: (3078, 23)
   TransactionID CustomerID   Restaurant  Location       Date      Time  \
0          10001   CUST0093     TacoTime   Chicago 2024-11-02  11:18:00   
1          10002   CUST0089      PizzaCo  New York 2024-11-02  13:27:00   
2          10003   CUST0092     TacoTime   Houston 2024-11-02  18:34:00   
3          10004   CUST0014  BurgerPlace   Houston 2024-11-02  12:15:00   
4          10005   CUST0089      PizzaCo  New York 2024-11-02  12:10:00   

    OrderType PaymentMethod  \
0     Dine-in     Gift Card   
1    Delivery          Cash   
2     Dine-in    Mobile App   
3     Takeout   Credit Card   
4  Drive-thru   Credit Card   

                                        ItemsOrdered  Subtotal  ...  \
0                         Beans, Quesadilla, Burrito     19.47  ...   
1  Veggie Pizza, Chicken Wings, Meat Lovers Pizza...     51.96  ...   
2                                 Beef Taco, Burrito     12.98  ...   
3                                       Cheeseburger

In [50]:
print("\nData Types: ")
print(df.dtypes)


Data Types: 
TransactionID                     int64
CustomerID                       object
Restaurant                       object
Location                         object
Date                     datetime64[ns]
Time                             object
OrderType                        object
PaymentMethod                    object
ItemsOrdered                     object
Subtotal                        float64
Discount                        float64
Tax                             float64
Tip                             float64
DeliveryFee                     float64
Total                           float64
ProcessingTimeMin               float64
EmployeeID                       object
Weather                          object
PromotionCode                    object
SatisfactionRating              float64
TransactionType                  object
RefundReason                     object
OriginalTransactionID           float64
dtype: object


In [51]:
print(df.isnull().sum())

TransactionID               0
CustomerID                  0
Restaurant                  0
Location                    0
Date                        0
Time                        0
OrderType                   0
PaymentMethod               0
ItemsOrdered                0
Subtotal                    0
Discount                    0
Tax                         0
Tip                         0
DeliveryFee                 0
Total                       0
ProcessingTimeMin          11
EmployeeID                 10
Weather                     9
PromotionCode            2497
SatisfactionRating       2067
TransactionType             0
RefundReason             2917
OriginalTransactionID    2917
dtype: int64


In [52]:
print("\nUnique Restaurants: ")
print(df['Restaurant'].unique())
print("\nUnique Payment Methods:")
print(df['PaymentMethod'].unique())


Unique Restaurants: 
['TacoTime' 'PizzaCo' 'BurgerPlace' 'SandwichShop' 'BurgerPlace.'
 'Pizza Co' 'Pizza Co.' 'Burger Place' 'Burger_Place' 'PizzaCo.']

Unique Payment Methods:
['Gift Card' 'Cash' 'Mobile App' 'Credit Card' 'Debit Card' 'GIFT CARD'
 'credit card' 'gift card' 'cash' 'MOBILE APP' 'mobile app' 'CASH'
 'debit card' 'DEBIT CARD' 'CREDIT CARD']


# Data Quality Issues Added

This updated code includes several realistic data quality issues that you'll need to clean:

1. **Refund Transactions**:
   * Added refund transactions (about 5% of transactions) with negative amounts
   * Linked to original transactions via OriginalTransactionID
   * Only applied to credit card transactions

2. **Inconsistent Formatting**:
   * Restaurant names have variations (e.g., "PizzaCo", "Pizza Co", "Pizza Co.")
   * Payment methods have inconsistent capitalization

3. **Missing Values**:
   * Some rows have NULL values for Weather, EmployeeID, or ProcessingTimeMin

4. **Duplicate Transactions**:
   * About 1% of transactions are duplicated with new transaction IDs

5. **Calculation Errors**:
   * Some refund transactions have slight discrepancies between components and totals

In [53]:
# Step 3: Clean Data

# 3a: Standardize restaurant names
print(df['Restaurant'].value_counts())

# Create mapping dictionary
restaurant_mapping = {
    "Pizza Co" : "PizzaCo",
    "Pizza Co." : "PizzaCo",
    'PizzaCo.' : 'PizzaCo',
    'Burger Place' : 'BurgerPlace',
    'BurgerPlace.' : 'BurgerPlace',
    'Burger_Place' : 'BurgerPlace'
}

df['Restaurant'] = df['Restaurant'].replace(restaurant_mapping)

print(df['Restaurant'].value_counts())

Restaurant
PizzaCo         821
TacoTime        792
SandwichShop    746
BurgerPlace     688
Burger Place      9
Pizza Co          7
BurgerPlace.      5
Pizza Co.         4
Burger_Place      4
PizzaCo.          2
Name: count, dtype: int64
Restaurant
PizzaCo         834
TacoTime        792
SandwichShop    746
BurgerPlace     706
Name: count, dtype: int64


In [54]:
# 3b: normalize payment methods
print(df['PaymentMethod'].value_counts())

payment_mapping = {
    'gift card' : 'Gift Card',
    'credit card' : 'Credit Card',
    'GIFT CARD' : 'Gift Card',
    'MOBILE APP' : 'Mobile App',
    'CREDIT CARD' : 'Credit Card',
    'debit card' : 'Debit Card',
    'mobile app' : 'Mobile App',
    'CASH' : 'Cash',
    'cash' : 'Cash',
    'DEBIT CARD' : 'Debit Card'
}

df['PaymentMethod'] = df['PaymentMethod'].replace(payment_mapping)
print("\nAfter replacement: ")
print(df['PaymentMethod'].value_counts())



PaymentMethod
Credit Card    711
Debit Card     619
Cash           566
Gift Card      562
Mobile App     529
gift card       15
credit card     14
GIFT CARD       11
MOBILE APP      10
CREDIT CARD      9
debit card       8
mobile app       7
CASH             7
cash             5
DEBIT CARD       5
Name: count, dtype: int64

After replacement: 
PaymentMethod
Credit Card    734
Debit Card     632
Gift Card      588
Cash           578
Mobile App     546
Name: count, dtype: int64


In [55]:
# 3c: Missing values
missing_cols = df.columns[df.isnull().any()].tolist()
print(f"Columns with missing values: {missing_cols}")

# Strategy for missing values
if 'Weather' in missing_cols:
    # Fill in missing weather with most common weather
    most_common_weather = df['Weather'].mode()[0]
    df['Weather'] = df['Weather'].fillna(most_common_weather)
    print('Filled missing weather with most common weather: ', most_common_weather)

if 'EmployeeID' in missing_cols:
    # Fill in missing employee ID with Unknown
    df['EmployeeID'] = df['EmployeeID'].fillna('Unknown')
    print("Filled missing employee ID with Unknown")

if 'ProcessingTimeMin' in missing_cols:
    # Fill in processing tiime with median based on order type
    for order_type in df['OrderType'].unique():
        median_processing_time = df[df['OrderType'] ==order_type]['ProcessingTimeMin'].median()
        df.loc[(df['OrderType'] == order_type) & 
               (df['ProcessingTimeMin'].isnull()), 'ProcessingTimeMin'] = median_processing_time
        print(f"Filled Missing processing time for {order_type} with: {median_processing_time}")



Columns with missing values: ['ProcessingTimeMin', 'EmployeeID', 'Weather', 'PromotionCode', 'SatisfactionRating', 'RefundReason', 'OriginalTransactionID']
Filled missing weather with most common weather:  Sunny
Filled missing employee ID with Unknown
Filled Missing processing time for Dine-in with: 29.0
Filled Missing processing time for Delivery with: 42.0
Filled Missing processing time for Takeout with: 16.0
Filled Missing processing time for Drive-thru with: 9.0


In [56]:
# 3d: remove duplicates

# Consider transactions as duplicates if they have the same customer, restaurant,
# date, time, order type, items ordered, and total amount, but different transaction IDs
potential_duplicate_columns = [
    'CustomerID', 'Restaurant', 'Date', 'Time', 'OrderType', 
    'ItemsOrdered', 'Total'
]

# Create temp identifier for potential duplicates
df['DuplicateGroup'] = df[potential_duplicate_columns].apply(tuple, axis=1)

# Find duplicate groups
duplicate_groups = df['DuplicateGroup'].value_counts()
duplicate_groups = duplicate_groups[duplicate_groups > 1]

print(f"Number of potential duplicate transaction groups: {len(duplicate_groups)}")

if len(duplicate_groups) > 0:
    # Mark duplicates, keeping the first occurrence
    df['IsDuplicate'] = False
    
    for group in duplicate_groups.index:
        group_df = df[df['DuplicateGroup'] == group]
        duplicated_indices = group_df.index[1:]  # Keep the first one
        df.loc[duplicated_indices, 'IsDuplicate'] = True
    
    print(f"Number of marked duplicate transactions: {df['IsDuplicate'].sum()}")
    
    
    # Remove duplicates
    df_cleaned = df[~df['IsDuplicate']]
    print(f"Shape after removing duplicates: {df_cleaned.shape}")

# Drop the temporary column
df = df.drop('DuplicateGroup', axis=1)




Number of potential duplicate transaction groups: 30
Number of marked duplicate transactions: 30
Shape after removing duplicates: (3048, 25)


In [57]:
# 3e: validate transaction totals (check if total = subtotal - discount + tax + tip + delivery fee)

df['ExpectedTotal'] = (df['Subtotal'] - df['Discount'] + df['Tax'] + df['Tip'] + df['DeliveryFee']).round(2)

df['TotalDifference'] = (df['Total'] - df['ExpectedTotal']).round(2)

discrepancies = df[abs(df['TotalDifference']) > 0.01]
print(f"Number of transactions with total discrepancies: {len(discrepancies)}")

if len(discrepancies) > 0:
    print("\nSample of transactions with discrepancies:")
    print(discrepancies[['TransactionID', 'Total', 'ExpectedTotal', 'TotalDifference', 'TransactionType']])

    # Correct discrepancies
    df.loc[abs(df['TotalDifference']) > 0.01, 'Total'] = df['ExpectedTotal']
    print("\nDiscrepancies correctd")

# Recalculate the difference after corrections
df['NewTotalDifference'] = (df['Total'] - df['ExpectedTotal']).round(2)
# Check if any discrepancies remain
remaining_discrepancies = df[abs(df['NewTotalDifference']) > 0.01]
print(f"\nRemaining discrepancies: {len(remaining_discrepancies)}")

df = df.drop(['ExpectedTotal', 'TotalDifference', 'NewTotalDifference'], axis=1)

Number of transactions with total discrepancies: 24

Sample of transactions with discrepancies:
      TransactionID  Total  ExpectedTotal  TotalDifference TransactionType
69            10070 -32.01         -32.07             0.06          Refund
205           10206 -35.40         -35.28            -0.12          Refund
218           10219 -34.60         -34.07            -0.53          Refund
350           10351 -22.97         -23.15             0.18          Refund
363           10364 -33.74         -34.07             0.33          Refund
374           10375 -29.45         -29.77             0.32          Refund
455           10456 -47.51         -47.75             0.24          Refund
564           10565  -7.96          -8.10             0.14          Refund
600           10601 -23.76         -23.74            -0.02          Refund
705           10706 -34.35         -34.61             0.26          Refund
996           10997  -8.35          -8.42             0.07          Refund
1309

In [58]:
# 3f: handle refunds
refunds = df[df['Total'] < 0]
print(f"Number of refund transactions: {len(refunds)}")

print(refunds[['TransactionID', 'Total', 'OriginalTransactionID']].head())

# Check if refunds are linked to orig transactions
linked_refunds = refunds[refunds['OriginalTransactionID'].notna()]
print(f"\nRefunds with links to original transactions: {len(linked_refunds)}")

payment_methods_for_refunds = refunds['PaymentMethod'].unique()
print(f"\nPayment methods for refunds: {payment_methods_for_refunds}")

# Add a flag column to easily identify refunds in analysis
df['IsRefund'] = df['TransactionType'] == 'Refund'

# remove refunds

# Get IDs of original transactions that were refunded
original_ids = df[df['IsRefund']]['OriginalTransactionID'].tolist()

# Filter out both refunds and their original transactions
df = df[~(df['IsRefund'] | df['TransactionID'].isin(original_ids))].copy()

# Print summary of removals
print(f"Removed {df['IsRefund'].sum()} refund transactions")
print(f"Removed {len(original_ids)} original transactions that were refunded")
print(f"Dataset size after: {len(df)}")

Number of refund transactions: 161
    TransactionID  Total  OriginalTransactionID
53          10054  -7.17                10027.0
62          10063 -17.45                10043.0
69          10070 -32.07                10016.0
87          10088 -37.02                10033.0
91          10092 -13.24                10050.0

Refunds with links to original transactions: 161

Payment methods for refunds: ['Credit Card']
Removed 0 refund transactions
Removed 161 original transactions that were refunded
Dataset size after: 2799


In [59]:
# Convert Date and Time Columns

df['Date'] = pd.to_datetime(df['Date'])

# Create a datetime column combining date and time
df['DateTime'] = pd.to_datetime(
    df['Date'].dt.strftime('%Y-%m-%d') + ' ' + 
    df['Time'].astype(str),
    errors='coerce'  # Handle any conversion errors
)

# Extract useful components
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfWeek'] = df['Date'].dt.day_name()
df['Hour'] = pd.to_datetime(df['Time'].astype(str)).dt.hour


/var/folders/31/b1v52j156gbfz83pmrnth3jc0000gn/T/ipykernel_69396/138550983.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Hour'] = pd.to_datetime(df['Time'].astype(str)).dt.hour


In [60]:
# Save Cleaned Data

# Save the cleaned data to a new file
df.to_excel('restaurant_transactions_cleaned.xlsx', index=False)
print(f"Cleaned data saved with {len(df)} rows and {len(df.columns)} columns.")

Cleaned data saved with 2799 rows and 31 columns.


In [61]:
# Verify the cleaning was successful
print("\nFinal data quality check:")

# Check for any remaining issues
print(f"Missing values after cleaning: {df.isnull().sum().sum()}")
print(f"Unique restaurant names: {df['Restaurant'].unique()}")
print(f"Unique payment methods: {df['PaymentMethod'].unique()}")
print(f"Refund transactions: {df['IsRefund'].sum()}")

# Basic summary stats on key metrics
print("\nSummary statistics after cleaning:")
print(df.groupby('Restaurant')['Total'].agg(['count', 'sum', 'mean', 'median']))

print(df.shape)


Final data quality check:
Missing values after cleaning: 9738
Unique restaurant names: ['TacoTime' 'PizzaCo' 'SandwichShop' 'BurgerPlace']
Unique payment methods: ['Gift Card' 'Cash' 'Mobile App' 'Credit Card' 'Debit Card']
Refund transactions: 0

Summary statistics after cleaning:
              count       sum       mean  median
Restaurant                                      
BurgerPlace     662  14949.64  22.582538  21.625
PizzaCo         752  25646.24  34.104043  33.510
SandwichShop    685  13017.10  19.003066  18.260
TacoTime        700  10417.50  14.882143  14.040
(2799, 31)
